# Experimentos de Métodos de Optimización

Notebook generado automáticamente a partir del script para comparar:
- Método de máximo descenso con Armijo
- Método de región de confianza con punto de Cauchy

Ejemplo de uso en una celda de código:
```python
import experiments_mo_jupyter as mo
df = mo.run_all()
df
```


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Script de experimentos numéricos para la función:
    f(x, y) = (exp(x^2) + y^4) / (y^2 + 1)

- Métodos:
    * Máximo descenso con búsqueda de línea tipo Armijo
    * Región de confianza con punto de Cauchy
- Experimentos:
    * 500 puntos iniciales ~ U([-100, 100]^2)
    * Estadísticas globales para comparar convergencia de ambos métodos

Salidas principales:
    - random_results_500.csv     (resultados por punto inicial)
    - random_stats_500.csv       (estadísticas globales por método)
"""

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import pandas as pd
from math import isfinite
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401  (para activar 3D)


# ============================================================
# Definición simbólica de f, gradiente y Hessiano
# ============================================================

x_sym, y_sym = sp.symbols('x y')
f_sym = (sp.exp(x_sym**2) + y_sym**4) / (y_sym**2 + 1)

grad_sym = [sp.diff(f_sym, x_sym), sp.diff(f_sym, y_sym)]
hess_sym = sp.hessian(f_sym, (x_sym, y_sym))

f_scalar = sp.lambdify((x_sym, y_sym), f_sym, 'numpy')
grad_scalar = sp.lambdify((x_sym, y_sym), grad_sym, 'numpy')
hess_scalar = sp.lambdify((x_sym, y_sym), hess_sym, 'numpy')


def f_xy(x, y):
    """f(x, y) para escalares o arrays de numpy."""
    return f_scalar(x, y)


def f_vec(x):
    """f para vector x = [x, y]."""
    return float(f_scalar(x[0], x[1]))


def grad_vec(x):
    """Gradiente como vector numpy de tamaño 2."""
    gx, gy = grad_scalar(x[0], x[1])
    return np.array([float(gx), float(gy)], dtype=float)


def hess_mat(x):
    """Hessiano 2x2 como matriz numpy."""
    H = np.array(hess_scalar(x[0], x[1]), dtype=float)
    return H


# ============================================================
# Utilidades numéricas
# ============================================================

def safe_f(x):
    """Evalúa f(x) y devuelve +inf si no es finita."""
    try:
        val = f_vec(x)
        if not np.isfinite(val):
            return np.inf
        return val
    except Exception:
        return np.inf


# ============================================================
# Método de Máximo Descenso (Steepest Descent) con Armijo
# ============================================================

def steepest_descent(x0,
                     alpha0=1.0,
                     beta=0.5,
                     sigma=1e-4,
                     max_iters=200,
                     tol_grad=1e-6):
    """
    Máximo descenso con búsqueda de línea tipo Armijo.

    Parámetros
    ----------
    x0 : iterable de longitud 2
        Punto inicial.
    alpha0 : float
        Paso inicial para la búsqueda de línea.
    beta : float
        Factor de reducción (0 < beta < 1).
    sigma : float
        Parámetro de Armijo (0 < sigma < 1).
    max_iters : int
        Máximo de iteraciones.
    tol_grad : float
        Tolerancia sobre la norma del gradiente.

    Returns
    -------
    traj : lista de np.ndarray
        Secuencia de puntos x_k.
    hist : pd.DataFrame
        Historial con columnas: k, fx, ng, alpha.
    """
    xk = np.array(x0, dtype=float)
    traj = [xk.copy()]
    rows = []

    for k in range(max_iters):
        fx = safe_f(xk)
        gk = grad_vec(xk)
        ng = np.linalg.norm(gk, ord=2)

        rows.append({"k": k, "fx": fx, "ng": ng, "alpha": np.nan})

        if not np.isfinite(fx) or ng < tol_grad:
            break

        pk = -gk
        alpha = alpha0

        # Búsqueda de línea de Armijo
        while True:
            x_new = xk + alpha * pk
            f_new = safe_f(x_new)

            if not np.isfinite(f_new):
                # forzar reducción si explota numéricamente
                alpha *= beta
            else:
                if f_new <= fx + sigma * alpha * np.dot(gk, pk):
                    break
                alpha *= beta

            if alpha < 1e-12:
                break

        rows[-1]["alpha"] = alpha

        if alpha < 1e-12 or not np.isfinite(f_new):
            # No se pudo encontrar un paso aceptable
            break

        xk = x_new
        traj.append(xk.copy())

    hist = pd.DataFrame(rows)
    return traj, hist


# ============================================================
# Método de Región de Confianza (punto de Cauchy)
# ============================================================

def trust_region(x0,
                 Delta0=1.0,
                 Delta_max=100.0,
                 eta=0.15,
                 max_iters=200,
                 tol_grad=1e-6):
    """
    Método de región de confianza con punto de Cauchy.

    Parámetros
    ----------
    x0 : iterable de longitud 2
        Punto inicial.
    Delta0 : float
        Radio inicial de la región de confianza.
    Delta_max : float
        Radio máximo permitido.
    eta : float
        Umbral de aceptación del paso (0 < eta < 0.25 típicamente).
    max_iters : int
        Máximo de iteraciones.
    tol_grad : float
        Tolerancia sobre la norma del gradiente.

    Returns
    -------
    traj : lista de np.ndarray
        Secuencia de puntos x_k.
    hist : pd.DataFrame
        Historial con columnas: k, fx, ng, Delta, rho.
    """
    xk = np.array(x0, dtype=float)
    Delta = Delta0
    traj = [xk.copy()]
    rows = []

    for k in range(max_iters):
        fx = safe_f(xk)
        gk = grad_vec(xk)
        Hk = hess_mat(xk)
        ng = np.linalg.norm(gk, ord=2)

        rows.append({"k": k, "fx": fx, "ng": ng, "Delta": Delta, "rho": np.nan})

        if not np.isfinite(fx) or ng < tol_grad:
            break

        if ng == 0:
            break

        # Punto de Cauchy
        gHg = float(np.dot(gk, Hk @ gk))
        if gHg <= 0:
            tau = 1.0
        else:
            tau = min(ng**3 / (Delta * gHg), 1.0)

        pk = -(tau * Delta / ng) * gk

        f_trial = safe_f(xk + pk)

        # Reducciones real y predicha
        ared = fx - f_trial
        pred = -(np.dot(gk, pk) + 0.5 * np.dot(pk, Hk @ pk))

        if pred <= 0 or not np.isfinite(pred):
            rho = -np.inf
        else:
            rho = ared / pred

        rows[-1]["rho"] = rho

        # Actualización de región de confianza
        if rho < 0.25:
            Delta *= 0.25
        else:
            if rho > 0.75 and np.isclose(np.linalg.norm(pk), Delta, rtol=1e-4, atol=1e-6):
                Delta = min(2.0 * Delta, Delta_max)

        # Aceptación / rechazo del paso
        if rho > eta and np.isfinite(f_trial):
            xk = xk + pk
            traj.append(xk.copy())
        # Si no se acepta, solo se reduce Delta y se reintenta en la próxima iteración

    hist = pd.DataFrame(rows)
    return traj, hist
# ============================================================
# Figuras para la función y la convergencia
# ============================================================

def plot_function_surface_and_contours(xmin=-3, xmax=3,
                                       ymin=-3, ymax=3,
                                       n_points=200,
                                       fname_surface="fig_f_surface.png",
                                       fname_contours="fig_f_contours.png"):
    """Genera una superficie 3D y un mapa de contornos de f en una ventana moderada."""
    xs = np.linspace(xmin, xmax, n_points)
    ys = np.linspace(ymin, ymax, n_points)
    X, Y = np.meshgrid(xs, ys)
    Z = f_xy(X, Y)
    # Evitar problemas de visualización con valores muy grandes
    Z = np.where(np.isfinite(Z), Z, np.nan)

    # Superficie 3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    ax.plot_surface(X, Y, Z, linewidth=0, antialiased=True)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("f(x,y)")
    ax.set_title("Superficie de f(x,y)")
    fig.tight_layout()
    fig.savefig(fname_surface, dpi=300)
    plt.close(fig)

    # Contornos en 2D (recortando valores muy grandes)
    Z_clipped = Z.copy()
    # Limitar a un percentil para ver mejor el valle
    finite_vals = Z[np.isfinite(Z)]
    if finite_vals.size > 0:
        vmax = np.quantile(finite_vals, 0.95)
        Z_clipped = np.clip(Z_clipped, None, vmax)

    fig2, ax2 = plt.subplots()
    cs = ax2.contour(X, Y, Z_clipped, levels=20)
    ax2.clabel(cs, inline=True, fontsize=8)
    ax2.set_xlabel("x")
    ax2.set_ylabel("y")
    ax2.set_title("Contornos de f(x,y)")
    fig2.tight_layout()
    fig2.savefig(fname_contours, dpi=300)
    plt.close(fig2)


def plot_iterations_hist(csv_path="random_results_500.csv",
                         fname_hist_sd="fig_hist_sd_iters.png",
                         fname_hist_tr="fig_hist_tr_iters.png"):
    """Histogramas del número de iteraciones para SD y TR."""
    df = pd.read_csv(csv_path)

    # SD
    fig_sd, ax_sd = plt.subplots()
    ax_sd.hist(df["SD_iters"], bins=range(1, 202, 5), edgecolor="black")
    ax_sd.set_xlabel("Iteraciones (SD)")
    ax_sd.set_ylabel("Frecuencia")
    ax_sd.set_title("Histograma de iteraciones - Steepest Descent")
    fig_sd.tight_layout()
    fig_sd.savefig(fname_hist_sd, dpi=300)
    plt.close(fig_sd)

    # TR
    fig_tr, ax_tr = plt.subplots()
    ax_tr.hist(df["TR_iters"], bins=range(1, 202, 5), edgecolor="black")
    ax_tr.set_xlabel("Iteraciones (TR)")
    ax_tr.set_ylabel("Frecuencia")
    ax_tr.set_title("Histograma de iteraciones - Trust Region")
    fig_tr.tight_layout()
    fig_tr.savefig(fname_hist_tr, dpi=300)
    plt.close(fig_tr)


def plot_r0_vs_iters(csv_path="random_results_500.csv",
                     fname_scatter="fig_r0_vs_iters.png"):
    """
    Dispersión radio inicial ||x0|| vs iteraciones de SD y TR.
    Ayuda a visualizar cómo empeora la convergencia al alejarse del valle.
    """
    df = pd.read_csv(csv_path)
    r0 = np.sqrt(df["x0_x"]**2 + df["x0_y"]**2)

    fig, ax = plt.subplots()
    ax.scatter(r0, df["SD_iters"], s=10, alpha=0.6, label="SD")
    ax.scatter(r0, df["TR_iters"], s=10, alpha=0.6, label="TR")
    ax.set_xlabel(r"$\|x_0\|$")
    ax.set_ylabel("Iteraciones")
    ax.set_title("Iteraciones vs distancia inicial")
    ax.legend()
    fig.tight_layout()
    fig.savefig(fname_scatter, dpi=300)
    plt.close(fig)


# ============================================================
# Experimentos aleatorios y estadísticas globales
# ============================================================

def run_random_experiments(n_samples=500,
                           low=-100.0,
                           high=100.0,
                           seed=123,
                           csv_path="random_results_500.csv",
                           stats_path="random_stats_500.csv"):
    """
    Corre SD y TR desde n_samples puntos iniciales ~ U([low, high]^2)
    y guarda:
      - resultados individuales en csv_path
      - estadísticas globales en stats_path

    Devuelve
    --------
    df : pd.DataFrame
        Resultados por punto inicial.
    stats : pd.DataFrame
        Estadísticas globales por método.
    """
    rng = np.random.default_rng(seed)
    inits = rng.uniform(low, high, size=(n_samples, 2))

    results = []

    for i, x0 in enumerate(inits):
        x0_tuple = (float(x0[0]), float(x0[1]))

        # ------- Máximo descenso -------
        traj_sd, hist_sd = steepest_descent(x0_tuple, alpha0=1.0)
        if len(hist_sd) > 0:
            sd_last = hist_sd.iloc[-1]
            sd_iters = int(sd_last["k"]) + 1
            sd_fx = float(sd_last["fx"])
            sd_ng = float(sd_last["ng"])
            sd_x_final = traj_sd[-1].tolist()
        else:
            sd_iters = np.nan
            sd_fx = np.nan
            sd_ng = np.nan
            sd_x_final = [np.nan, np.nan]

        # ------- Región de confianza -------
        traj_tr, hist_tr = trust_region(x0_tuple, Delta0=1.0)
        if len(hist_tr) > 0:
            tr_last = hist_tr.iloc[-1]
            tr_iters = int(tr_last["k"]) + 1
            tr_fx = float(tr_last["fx"])
            tr_ng = float(tr_last["ng"])
            tr_x_final = traj_tr[-1].tolist()
        else:
            tr_iters = np.nan
            tr_fx = np.nan
            tr_ng = np.nan
            tr_x_final = [np.nan, np.nan]

        results.append({
            "sample_id": i,
            "x0_x": x0_tuple[0],
            "x0_y": x0_tuple[1],
            "SD_iters": sd_iters,
            "TR_iters": tr_iters,
            "SD_fx_final": sd_fx,
            "TR_fx_final": tr_fx,
            "SD_grad_norm_final": sd_ng,
            "TR_grad_norm_final": tr_ng,
            "SD_x_final": sd_x_final,
            "TR_x_final": tr_x_final,
        })

    df = pd.DataFrame(results)
    df.to_csv(csv_path, index=False)

    # ---------- Estadísticas globales para análisis de convergencia ----------
    sd_cols = ["SD_iters", "SD_fx_final", "SD_grad_norm_final"]
    tr_cols = ["TR_iters", "TR_fx_final", "TR_grad_norm_final"]

    sd_stats = df[sd_cols].agg(["mean", "std", "min", "max"])
    tr_stats = df[tr_cols].agg(["mean", "std", "min", "max"])

    sd_stats.columns = ["iters", "f_final", "grad_norm_final"]
    tr_stats.columns = ["iters", "f_final", "grad_norm_final"]

    stats = pd.concat(
        {"SteepestDescent": sd_stats, "TrustRegion": tr_stats},
        axis=1
    )

    stats.to_csv(stats_path)

    print("\n=== Estadísticas globales ({} muestras) ===".format(n_samples))
    print(stats)

    return df, stats


# ============================================================
# Punto de entrada principal
# ============================================================

if __name__ == "__main__":
    # Ejecutar experimentos aleatorios
    df_rand, stats_rand = run_random_experiments(
        n_samples=500,
        low=-100.0,
        high=100.0,
        seed=123,
        csv_path="random_results_500.csv",
        stats_path="random_stats_500.csv",
    )

    # Generar figuras para incluir en el informe
    plot_function_surface_and_contours()
    plot_iterations_hist("random_results_500.csv")
    plot_r0_vs_iters("random_results_500.csv")

    print("\nFiguras guardadas como:")
    print(" - fig_f_surface.png")
    print(" - fig_f_contours.png")
    print(" - fig_hist_sd_iters.png")
    print(" - fig_hist_tr_iters.png")
    print(" - fig_r0_vs_iters.png")

